# Google Play Movie Crawler

In [66]:
import requests as r
from bs4 import BeautifulSoup
import json
import re
import pymongo
from pymongo import MongoClient
import time
import random

client = MongoClient("10.120.28.17", 27017)

#我們自己訂的db 和 collection
db = client.googleplay
collect = db.googleplay_test2

In [67]:
def create_payload(url, querystring, headers):
    """ This is the function for create payload for google_play_movie()  to use
          arg = url, querystring, headers from google_play_movie()
          return  = 'payload_list'
    """
    print "111"
    # get first movie url in the page for further compare with the urls get from payload
    payload_first = "-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"start\"\r\n\r\n0\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"num\"\r\n\r\n60\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"numChildren\"\r\n\r\n0\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"ipf\"\r\n\r\n1\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"xhr\"\r\n\r\n1\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"h1\"\r\n\r\nzh_TW\r\n-----011000010111000001101001--"
    url_first = url
    print url_first
    querystring_first = querystring
    print querystring_first
    headers_first = headers
    print headers_first
#    proxies_list = ["220.129.196.210:443", "114.40.169.86:443"]
#    proxies = {
#        'https': random.choice(proxies_list)
#    }
    response_first = r.request("POST", url_first, data=payload_first, headers=headers_first, params=querystring_first)
    print response_first
    response_first.encoding = 'utf8'
    url_soup_first = BeautifulSoup(response_first.text, 'lxml')
    first = url_soup_first.find('a', class_="title")
    print first
    
    i = 0
    payload_list = []
    
    while( i > -1 ):
        # create payload for request, from o, get 60 posts each time
        payload = "-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"start\"\r\n\r\n0\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"num\"\r\n\r\n60\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"numChildren\"\r\n\r\n0\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"ipf\"\r\n\r\n1\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"xhr\"\r\n\r\n1\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"h1\"\r\n\r\nzh_TW\r\n-----011000010111000001101001--"
        url_forPayLoad = url
        querystring = querystring
        headers = headers
#    proxies_list = ["220.129.196.210:443", "114.40.169.86:443"]
#    proxies = {
#        'https': random.choice(proxies_list)
#    }
        response = r.request("POST", url_forPayLoad, data=payload, headers=headers, params=querystring)
        response.encoding = 'utf8'
        url_soup = BeautifulSoup(response.text, 'lxml')
#        print url_soup
        # for checking if  get repeat url, get first url to campare
        lists_first = url_soup.find('a', class_="title")
#        print lists_first
        # in some case, first url might be empty if payload is out of range
        if lists_first == None:
            return payload_list
        else:            
#        print lists
            # for the first request, append lists anyway
            if i == 0:
                payload_list.append(payload)
                i += 60
            # if first url = first or empty, stop the loop and return the payload lists -> this means payload is outof range, so need to stop
            elif lists_first == first or lists_first == '' '':
                return payload_list
            # others append lists
            else:
                payload_list.append(payload)
                i += 60
    # return the lists of payload
    return payload_list

In [68]:
def url_lists(response):
    """ This is the founction for getting each movie's url from a payload list        
          arg = "response"
          return = url_lists [
                          url,
                          url
                          ......          ]
    
    """
    url_soup = BeautifulSoup(response.text, 'lxml')
    lists = url_soup.find_all('a', class_="title")
    url_lists = []
    # for a single response it will contain 60 movie urls
    for url in lists:
        url_lists.append(url['href'])
#        print len(url_lists)
    return url_lists

In [69]:
def movie_basic_info(url, playid):
    """ This is the function for each movie's basic information
          arg = url_lists
          return = movie_info_list [title, total_star,  total_num]
    """
    movie_info_list = []
    
    url = url
    querystring = {"id":playid , "authuser":"0"}
    payload = "-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"ipf\"\r\n\r\n1\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"xhr\"\r\n\r\n1\r\n-----011000010111000001101001--"
    headers = {
        'content-type': "multipart/form-data; boundary=---011000010111000001101001",
        'cache-control': "no-cache",
#        'postman-token': "9bfafe06-84ee-ed77-bc09-8df3c9f44137"
        }
#    proxies_list = ["220.129.196.210:443", "114.40.169.86:443"]
#    proxies = {
#        'https': random.choice(proxies_list)
#    }
    response = r.request("POST", url, data=payload, headers=headers, params=querystring)
    response.encoding = 'utf-8'
    content_soup = BeautifulSoup(response.text, 'lxml')
    
    # get each movie's title & total_star & total_num
    try:
        total_star = content_soup.find('div', class_="tiny-star")['aria-label']
        total_num = content_soup.find('span', class_="reviewers-small")['aria-label']
        title = content_soup.find(class_="document-title").text.strip()

        movie_info_list.append(title)
        movie_info_list.append(total_star)
        movie_info_list.append(total_num)
#        print movie_info_list
    # if there were no one comment, there's no total star. catch the exception, and given the total_star=0 & total_num=0
    except:
        total_star = "0"
        total_num = "0"
        title = content_soup.find(class_="document-title").text.strip()
        
        movie_info_list.append(title)
        movie_info_list.append(total_star)
        movie_info_list.append(total_num)
#        print movie_info_lists
    time.sleep(1)
    return movie_info_list

In [70]:
def movie_reviews(movie_id):
    """ This is the function for getting each movie's total reviews
          arg = movie_id # movie_id is for payload
          return = all_reviews = {
                      user : star,
                      user : star          }
    """
    single_movie_dict = {}
    
    url = "http://play.google.com/store/getreviews"
    querystring = {"authuser":"0"}
       
    headers = {
        'content-type': "multipart/form-data; boundary=---011000010111000001101001; charset=UTF-8",
        'cache-control': "no-cache",
#        'postman-token': "f69f4f7b-9d24-94c1-466f-91c4e72bde82",
        }
    
    page = 0
    while (page > -1):
        payload = "-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"reviewType\"\r\n\r\n0\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"pageNum\"\r\n\r\n" + str(page) + "\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"id\"\r\n\r\nmovie-" + str(movie_id) + "\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"reviewSortOrder\"\r\n\r\n4\r\n-----011000010111000001101001\r\nContent-Disposition: form-data; name=\"xhr\"\r\n\r\n1\r\n-----011000010111000001101001--"
        
#    proxies_list = ["220.129.196.210:443", "114.40.169.86:443"]
#    proxies = {
#        'https': random.choice(proxies_list)
#    }
        response = r.request("POST", url, data=payload, headers=headers, params=querystring)
        response.encoding = 'utf-8'
        
        #  if len(response.text.split(",")) == 2 means there's no commend, return an empty all_reviews
        if len(response.text.split(",")) == 2:
            return single_movie_dict
        
        # if response.text.split(",")[2] == "" means payload is out of range, return all_reviews
        elif response.text.split(",")[2] == u'""':
            return single_movie_dict
        
        # else is the normal reviews 
        else:
            # response is a json like document, replace unicode to characters fro further BeautifulSoup
            a = response.text.replace("\u003c", "<").replace("\u003d", "=").replace("\u003e", ">").replace('\\"', '"')
            soup = BeautifulSoup(a[17:-7], 'lxml')
 
            # get all reviews from one request
            single_review = soup.find_all('div', class_="review-info")
            # get each review
            for single in single_review:
                user = single.find('span', class_="author-name").text.replace(".", " ")
                star_long = single.find('div', class_="tiny-star")["aria-label"]
                star = star_long.split(" ")[1]
                single_movie_dict.update({user : star})                
            page += 1 # page plus one for next page of reviews
        time.sleep(3)
    return single_movie_dict

In [71]:
def movie_info(url_lists, num):
    """ This is the function for each movie's review
          arg = url_lists
          return = 
          OutPut = one json for one movie to Mongo DB
                          movie = { "title" : "", "type" : "", "rating": "", "total_num": "",  "reviews" : "" }
    """
    now = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    url_left = "http://play.google.com"    
    category = num
    count = 1
    
    # run each movie url
    for single_url in url_lists:
        
        movie = {}
        
        # total_url is create for request
        total_url = url_left + single_url.split("?")[0]
        
        # movie_id is create for querystring
        movie_id = str(single_url.split("?")[1].split("=")[1])
        
        # call movie_basic_info() for movie's basic information, get a list of [title, total_star,  total_num]
        basic = movie_basic_info(total_url, movie_id)
        
        # call movie_reviews() function for total reviews, get a dict of user reviews
        total_reviews = movie_reviews(movie_id)
        
        # insert data into movie
        movie.update({"title" : basic[0]})
        movie.update({"type" : category})
        movie.update({"rating" : basic[1]})
        movie.update({"total_num" : basic[2]})
        movie.update({"reviews" : total_reviews})
        
        # output movie into MongoDB
        collect.insert_one(movie)
        print now + '\t' + basic[0] + '\n' + single_url
        print count
    
        # count for total number of output
        count += 1
    
    time.sleep(1)
    return count

In [72]:
def google_play_movie(category):
    """ This is the main function to crawle google play movie
          arg = "num" #catogory number
          return = "Crawler finised! with total insert number"
    """
    # lists for 熱門電影 urls
    top_urls = []
    # lists for 新片上架 urls
    mo_urls = []
    number_insert_top = 0
    number_insert_mo = 0
    
    for num in category:
        
        # 熱門電影 = top
        url_top = "http://play.google.com/store/movies/category/" + str(num) + "/collection/topselling_paid"
        # 新片上架 = mo
        url_mo = "http://play.google.com/store/movies/category/" + str(num) + "/collection/movers_shakers"

        querystring = {"authuser":"0"}
        headers = {
            'content-type': "multipart/form-data; boundary=---011000010111000001101001",
            'cache-control': "no-cache",
            'postman-token': "f5c415de-8ec3-1c1b-28bf-0598323cfa8f"
            }
#    proxies_list = ["220.129.196.210:443", "114.40.169.86:443"]
#    proxies = {
#        'https': random.choice(proxies_list)
#    }
        
        # for payload, call create_payload() function
        payload_list_top = create_payload(url_top, querystring, headers)
        payload_list_mo = create_payload(url_mo, querystring, headers)
        print payload_list_top
        
        # use payload to request movies
        for payload_top in payload_list_top:
            response_top = r.request("POST", url_top, data=payload_top, headers=headers, params=querystring)
            response_top.encoding = 'utf8'
            # call url_lists() function to get urls from each response
            top_url_lists = url_lists(response_top)
            # call movie_info() function to get all informations for each url, !!!OutPut to MongoDB by this function!!!
            number_insert_top = movie_info(top_url_lists, num)
        
        # same for the mo
        for payload_mo in payload_list_mo:
            response_mo = r.request("POST", url_mo, data=payload_mo, headers=headers, params=querystring)
            response_mo.encoding = 'utf8'
            mo_url_lists = url_lists(response_mo)
            number_insert_mo = movie_info(mo_url_lists, num)
        
        # count for the total number of movies insert into MongoDB
        total_insert_number = int(number_insert_top) + int(number_insert_mo)
    
    return "Crawler finised!" + str(total_insert_number)

In [73]:
google_play_movie([1])

111
http://play.google.com/store/movies/category/1/collection/topselling_paid
{'authuser': '0'}
{'postman-token': 'f5c415de-8ec3-1c1b-28bf-0598323cfa8f', 'content-type': 'multipart/form-data; boundary=---011000010111000001101001', 'cache-control': 'no-cache'}
<Response [400]>
None


ConnectionError: ('Connection aborted.', error(10060, '\xb3s\xbdu\xb9\xc1\xb8\xd5\xa5\xa2\xb1\xd1\xa1A\xa6]\xac\xb0\xb3s\xbdu\xb9\xef\xb6H\xa6\xb3\xa4@\xacq\xae\xc9\xb6\xa1\xa8\xc3\xa5\xbc\xa5\xbf\xbdT\xa6^\xc0\xb3\xa1A\xa9\xce\xacO\xb3s\xbdu\xab\xd8\xa5\xdf\xa5\xa2\xb1\xd1\xa1A\xa6]\xac\xb0\xb3s\xbdu\xaa\xba\xa5D\xbe\xf7\xb5L\xaak\xa6^\xc0\xb3\xa1C'))